In [ ]:
import numpy as np

In [ ]:
seq1 = "ATGCAGCAGCAGCCA"
seq2 = "ATATAT"

MATCH         =  1
MISMATCH      = -1
LINEAR_GAP    = -4
AFFINE_OPEN   = -10
AFFINE_EXTEND =  -1

def score_pair(a, b):
    """+1 если символы совпадают, -1 если нет"""
    return MATCH if a == b else MISMATCH

## Алгоритм Нидлмана-Вунша

Алгоритм глобального выравнивания двух последовательностей.  
Заполняем матрицу ДП размером `(len(seq1)+1) × (len(seq2)+1)`,  
выбирая в каждой клетке максимум из трёх вариантов:

| Вариант | Формула | Смысл |
|---|---|---|
| Диагональ | `dp[i-1][j-1] + score(a,b)` | выровнять символ к символу |
| Вверх | `dp[i-1][j] + gap` | гэп в seq2 |
| Влево | `dp[i][j-1] + gap` | гэп в seq1 |

После заполнения матрицы — **обратный ход** (traceback) из правого нижнего угла в левый верхний.

In [ ]:
def needleman_wunsch_linear(s1, s2, gap=LINEAR_GAP):
    n, m = len(s1), len(s2)

    # dp[i][j] — лучший счёт выравнивания s1[:i] и s2[:j]
    dp = np.zeros((n+1, m+1), dtype=int)
    # tb[i][j] — откуда пришли: 1=диаг, 2=сверху, 3=слева
    tb = np.zeros((n+1, m+1), dtype=int)

    # инициализация первого столбца и строки — только гэпы
    for i in range(1, n+1): dp[i][0] = i * gap;  tb[i][0] = 2
    for j in range(1, m+1): dp[0][j] = j * gap;  tb[0][j] = 3

    # заполнение матрицы
    for i in range(1, n+1):
        for j in range(1, m+1):
            diag = dp[i-1][j-1] + score_pair(s1[i-1], s2[j-1])
            up   = dp[i-1][j]   + gap
            left = dp[i][j-1]   + gap

            best = max(diag, up, left)
            dp[i][j] = best
            tb[i][j] = 1 if best == diag else (2 if best == up else 3)

    # обратный ход: идём из угла (n,m) в (0,0)
    a1, a2 = [], []
    i, j = n, m
    while i > 0 or j > 0:
        if tb[i][j] == 1:                        # диагональ — оба символа
            a1.append(s1[i-1]); a2.append(s2[j-1]); i -= 1; j -= 1
        elif tb[i][j] == 2:                      # сверху — гэп в seq2
            a1.append(s1[i-1]); a2.append('-');  i -= 1
        else:                                    # слева — гэп в seq1
            a1.append('-'); a2.append(s2[j-1]); j -= 1

    return dp, dp[n][m], ''.join(reversed(a1)), ''.join(reversed(a2))

## Аффинный штраф — три матрицы

Вместо одной матрицы используем три:

- **M[i][j]** — лучший счёт, если `s1[i]` и `s2[j]` выровнены символ к символу  
- **X[i][j]** — лучший счёт, если здесь **гэп в seq2** (пропуск символа seq1)  
- **Y[i][j]** — лучший счёт, если здесь **гэп в seq1** (пропуск символа seq2)

Идея: открыть гэп дорого (`open = -10`), но продолжать его дёшево (`extend = -1`).  
Это значит: лучше один длинный гэп, чем несколько коротких.

In [ ]:
def needleman_wunsch_affine(s1, s2, gap_open=AFFINE_OPEN, gap_ext=AFFINE_EXTEND):
    n, m = len(s1), len(s2)
    NEG_INF = -10**9  # «минус бесконечность» — недостижимое состояние

    M = np.full((n+1, m+1), NEG_INF, dtype=float)
    X = np.full((n+1, m+1), NEG_INF, dtype=float)
    Y = np.full((n+1, m+1), NEG_INF, dtype=float)

    # матрицы traceback для каждой из трёх матриц
    tb_M = [['']*( m+1) for _ in range(n+1)]
    tb_X = [['']*(m+1) for _ in range(n+1)]
    tb_Y = [['']*(m+1) for _ in range(n+1)]

    M[0][0] = 0
    # начальные гэпы: открываем один раз и продолжаем
    for i in range(1, n+1): X[i][0] = gap_open + gap_ext * i;  tb_X[i][0] = 'X'
    for j in range(1, m+1): Y[0][j] = gap_open + gap_ext * j;  tb_Y[0][j] = 'Y'

    for i in range(1, n+1):
        for j in range(1, m+1):
            s = score_pair(s1[i-1], s2[j-1])

            # M: приходим по диагонали из любой матрицы
            cM = {'M': M[i-1][j-1]+s, 'X': X[i-1][j-1]+s, 'Y': Y[i-1][j-1]+s}
            bM = max(cM, key=cM.get);  M[i][j] = cM[bM];  tb_M[i][j] = bM

            # X: гэп в seq2 — либо открываем новый, либо продолжаем
            cX = {'M': M[i-1][j]+gap_open+gap_ext,
                  'X': X[i-1][j]+gap_ext,
                  'Y': Y[i-1][j]+gap_open+gap_ext}
            bX = max(cX, key=cX.get);  X[i][j] = cX[bX];  tb_X[i][j] = bX

            # Y: гэп в seq1 — либо открываем новый, либо продолжаем
            cY = {'M': M[i][j-1]+gap_open+gap_ext,
                  'Y': Y[i][j-1]+gap_ext,
                  'X': X[i][j-1]+gap_open+gap_ext}
            bY = max(cY, key=cY.get);  Y[i][j] = cY[bY];  tb_Y[i][j] = bY

    # финальный счёт — максимум из трёх угловых клеток
    fs = {'M': M[n][m], 'X': X[n][m], 'Y': Y[n][m]}
    cur = max(fs, key=fs.get)

    # обратный ход
    a1, a2 = [], []
    i, j = n, m
    while i > 0 or j > 0:
        if cur == 'M':
            a1.append(s1[i-1]); a2.append(s2[j-1])
            cur = tb_M[i][j];  i -= 1;  j -= 1
        elif cur == 'X':
            a1.append(s1[i-1]); a2.append('-')
            cur = tb_X[i][j];  i -= 1
        else:
            a1.append('-'); a2.append(s2[j-1])
            cur = tb_Y[i][j];  j -= 1

    return M, X, Y, fs[max(fs, key=fs.get)], ''.join(reversed(a1)), ''.join(reversed(a2))

In [ ]:
def print_matrix(mat, s1, s2, title):
    """Выводит матрицу ДП с подписями строк и столбцов."""
    print(f"\n{title}")
    print("       " + "  ".join(f"{c:>4}" for c in ("-" + s2)))
    for i, row in enumerate(mat):
        label = ("-" + s1)[i]
        # заменяем -1000000000 на -∞ для читаемости
        vals = "  ".join("-∞  " if abs(v) > 999 else f"{int(v):>4}" for v in row)
        print(f"  {label}    {vals}")

def print_alignment(a1, a2, sc, title):
    """Выводит выравнивание: seq1 / разметка / seq2 и краткую статистику."""
    markup = ''.join(
        '|' if c1 == c2 else (' ' if '-' in (c1, c2) else '*')
        for c1, c2 in zip(a1, a2)
    )
    print(f"\n{'─'*45}")
    print(f"  {title}")
    print(f"  Score = {int(sc)}")
    print(f"{'─'*45}")
    print(f"  seq1:  {a1}")
    print(f"         {markup}")
    print(f"  seq2:  {a2}")
    print(f"  совпадений={markup.count('|')}  замен={markup.count('*')}  гэпов={markup.count(' ')}")

In [ ]:
dp_lin, score_lin, a1_lin, a2_lin = needleman_wunsch_linear(seq1, seq2)

print_matrix(dp_lin, seq1, seq2, f"Матрица ДП — линейный штраф (gap={LINEAR_GAP})")
print_alignment(a1_lin, a2_lin, score_lin, f"Выравнивание — линейный штраф (gap={LINEAR_GAP})")

In [ ]:
M_aff, X_aff, Y_aff, score_aff, a1_aff, a2_aff = needleman_wunsch_affine(seq1, seq2)

print_matrix(M_aff, seq1, seq2, f"Матрица M — совпадения/замены (open={AFFINE_OPEN}, ext={AFFINE_EXTEND})")
print_matrix(X_aff, seq1, seq2,  "Матрица X — гэп в seq2")
print_matrix(Y_aff, seq1, seq2,  "Матрица Y — гэп в seq1")
print_alignment(a1_aff, a2_aff, score_aff, f"Выравнивание — аффинный штраф (open={AFFINE_OPEN}, ext={AFFINE_EXTEND})")

In [ ]:
print("\n" + "="*45)
print("  Итог")
print("="*45)
print(f"  Линейный штраф (gap={LINEAR_GAP}):               Score = {score_lin}")
print(f"  Аффинный штраф (open={AFFINE_OPEN}, ext={AFFINE_EXTEND}):  Score = {int(score_aff)}")

## Вывод

**Линейный штраф (Score = −34)** штрафует каждый символ гэпа одинаково.  
Алгоритм распределяет гэпы по всему выравниванию — получается несколько коротких разрывов.

**Аффинный штраф (Score = −19)** делает открытие гэпа дорогим (−10),  
но продолжение — почти бесплатным (−1 за символ).  
Алгоритм предпочитает один длинный гэп вместо нескольких коротких —  
и итоговый score выше.

Это лучше отражает **биологическую реальность**:  
инсерция или делеция — это одно мутационное событие (например, ошибка репликации),  
и неважно, 3 нуклеотида выпало или 10.  
Аффинная модель не штрафует жёстко за длину такого блока,  
поэтому выравнивание получается биологически более правдоподобным.